In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

In [2]:
from operator import itemgetter

from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = AzureChatOpenAI(temperature=0.3,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

'El país en el que nació la ciudad de Honolulu, Hawái, donde nació Barack Obama, el 44º presidente de los Estados Unidos, es Estados Unidos.'

In [3]:
from langchain.schema.runnable import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [4]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Coral. and the flag of Comoros?')])

In [5]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color coral is a shade of orange with a pinkish undertone, resembling the color of the marine coral. \n\nThe flag of Comoros consists of four horizontal stripes of yellow, white, red, and blue, from top to bottom. In the upper hoist-side corner, there is a green triangle with a white crescent and four white stars inside it.')

        Input
         / \
        /   \
    Branch1 Branch2
        \   /
         \ /
        Combine

In [8]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | model
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | model
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | model
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | model
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [9]:
chain.invoke({"input": "crypto investing"})

'While there are potential benefits to crypto investing, it is important to acknowledge the risks and challenges associated with it. The high volatility, lack of regulation, potential for fraud, and technological vulnerabilities are all significant concerns that investors should be aware of. Additionally, the lack of intrinsic value and the complexity of choosing the right cryptocurrency add to the inherent risks of this investment. It is crucial for investors to exercise caution, conduct thorough research, and carefully consider their risk tolerance before venturing into the crypto market.'